# Init

In [22]:
import sys
import os
import shutil
import logging
# DEBUG: Detailed information, typically of interest only when diagnosing problems.
# INFO: Confirmation that things are working as expected.
# WARNING: An indication that something unexpected happened, or indicative of some problem in the near future (e.g. ‘disk space low’). The software is still working as expected.
# ERROR: Due to a more serious problem, the software has not been able to perform some function.
# CRITICAL: A serious error, indicating that the program itself may be unable to continue running.
import json
import pandas as pd
import numpy as np
import string
import random
import math
import time
import re
from datetime import datetime
from datetime import date
from datetime import timedelta
from datetime import timezone
from pytz import timezone
import pytz
import pause
import humanfriendly
from bs4 import BeautifulSoup
from tabulate import tabulate
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from flask import Flask, jsonify
from flask import Flask, session
from flask_session import Session
from werkzeug.wrappers import Request, Response
log_file='sample.log'
working_time_zone='Europe/Kiev'
alpari_date_format="%m/%d/%Y"
fh_time_format="%Y-%m-%d %H:%M:%S"
fh_date_format='%Y-%m-%d'
os.environ['TZ'] = working_time_zone
time.tzset()
work_dir=os.getcwd()
log_path=os.path.join(work_dir,log_file)
open(log_path, 'w').close()
os.chmod(log_path, 0o775)
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger('news_service')
logger.handlers = []
fh = logging.FileHandler(log_path)
fh.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(message)s', 
                              datefmt=fh_time_format)
fh.setFormatter(formatter)
ch.setFormatter(formatter)
logger.addHandler(fh)
logger.addHandler(ch)
logger.info('service started')
logger.info(work_dir)
logger.info("log path is " + log_path)
print(datetime.now())

2019-01-24 15:48:55:INFO:news_service:service started
2019-01-24 15:48:55:INFO:news_service:/home/jovyan/work/R/nginx-flask-plumber
2019-01-24 15:48:55:INFO:news_service:log path is /home/jovyan/work/R/nginx-flask-plumber/sample.log


2019-01-24 15:48:55.724366


In [23]:
app = Flask(__name__)
app.debug = True
SESSION_TYPE = 'filesystem'
app.config.from_object(__name__)
Session(app)

# Flask routes

In [24]:
@app.route('/echo')
def flask_echo():
    return "News service written in python is ready!\r\n"
@app.route('/sleep')
def flask_sleep():
    print('Sleeping!')
    time.sleep(10000)
    return "Sleep finished\r\n"
@app.route('/open_browser')
def flask_open_browser():
    driver, text=open_browser()
    session["driver"]=driver
    return "Browser opened\r\n" + text
@app.route('/close_browser')
def flask_close_browser():
    close_browser(session["driver"])
    return "Browser closed\r\n"
@app.route('/pause')
def flask_pause():
    print("Pause for 10 seconds")
    dt=datetime.now()+timedelta(seconds=10)
    pause.until(dt)
    return('Pause finished\r\n')
@app.route('/test_connectivity')
def flask_test_connectivity():
    print("Pause for 10 seconds")
    dt=datetime.now()+timedelta(seconds=10)
    pause.until(dt)
    return('Pause finished\r\n')

# Testing

In [49]:
def create_test_df():
    date_rng = pd.date_range(start='2019-01-24', end='2019-01-25', freq='H')
    l=len(date_rng)
    dts = pd.to_datetime(date_rng, format=fh_time_format)  
    hashes=[''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(3)) for x in range(l)]
    ns=list(range(l))
    vs=random.sample(range(100), l)
    d={'dt': dts, "event":hashes, "number": ns, "value": vs}
    df=pd.DataFrame(d)
    df['dt']=[dt.strftime('%Y-%m-%d %H:%M:%S') for dt in df.dt]
    return df

In [51]:
df=create_test_df()
df.head()
out_file='test.json'
df.to_json(out_file)
df1=pd.read_json(out_file)
df1.head()

,dt,event,number,value
0,2019-01-24 00:00:00,WNB,0,54
1,2019-01-24 01:00:00,HS5,1,79
10,2019-01-24 10:00:00,VQM,10,1
11,2019-01-24 11:00:00,TUB,11,6
12,2019-01-24 12:00:00,3DJ,12,97


In [52]:
in_file='response.json'
df=pd.read_json(in_file)
df.head()

,dt,event,m,number,value
0,2019-01-24 00:00:00,9H8,0,0,2
1,2019-01-24 01:00:00,5TO,21,1,21
2,2019-01-24 02:00:00,PE8,100,2,50
3,2019-01-24 03:00:00,QUI,195,3,65
4,2019-01-24 04:00:00,W5X,316,4,79


# Browser automation

In [4]:
def open_browser(sleep_time=10):
    print ('start opening browser')
    cap = DesiredCapabilities.CHROME
    driver = webdriver.Remote(command_executor='http://selenium-hub:4444/wd/hub',desired_capabilities=cap) 
    url="https://www.google.com"
    print ('Opening URL: ' + url + ' waiting approximately ' + str(sleep_time))
    driver.get(url)
    wait = WebDriverWait(driver, sleep_time)
    element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.gLFyf')))    
    print ('browser opened')
    text=driver.find_element_by_xpath("html").text
    print (text)
    return driver, text
def close_browser(driver):
    print ('closing chrome driver')
    driver.quit()
    print ('driver closed')

# Main

In [7]:
try:
    if __name__ == '__main__':
            app.run(host='0.0.0.0', port=5000) # open url http://0.0.0.0:5000/
except Exception as e:
    print('Exception starting flask app: ' + str(e))
pass 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Exception starting flask app: [Errno 98] Address already in use


In [18]:
!echo $GIT_VERSION